# KR1: Type Hierarchy
Types in Julia can be grouped together into supertypes (called abstract types), which can then be further grouped into another supertype, and so on, forming a Julia data type hierarchy. We can use the `supertypes()` and `subtypes()` functions to explore a data type's parent and child types. 

The `Any` type encompasses all most singular data types across Julia, among these is the `Number` type

In [1]:
supertypes(Number)

(Number, Any)

Inverstigating its subtypes,

In [2]:
subtypes(Number)

2-element Vector{Any}:
 Complex
 Real

In [3]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

In [4]:
subtypes(Integer)

3-element Vector{Any}:
 Bool
 Signed
 Unsigned

In [5]:
subtypes(Signed)

6-element Vector{Any}:
 BigInt
 Int128
 Int16
 Int32
 Int64
 Int8

In [6]:
subtypes(Int64)

Type[]

If a type has no further subtypes, they are considered as concrete types where data can be stored as that form.

# KR2: Structs
Structs are similar to Python classes. They can be regarded as "objects" containing attributes. Normally, a `struct` has immutable attributes/arguments (i.e. we cannot change them in the future).

In [7]:
struct Pokemon
    name::String
    level::Int32
    power::Float64
end

In [8]:
pikachu = Pokemon("Pikachu",25,65.3)

Pokemon("Pikachu", 25, 65.3)

In [9]:
pikachu.level

25

In [10]:
# Attempting to alter Pikachu's level results in an error since Pokemon is immutable
pikachu.level = 30

LoadError: setfield! immutable struct of type Pokemon cannot be changed

If we want alterable attributes, we can use `mutable struct` instead

In [11]:
mutable struct m_Pokemon
    name::String
    level::Int32
    power::Float64    
end

In [12]:
togepi = m_Pokemon("Togepi",15,12.5)

m_Pokemon("Togepi", 15, 12.5)

In [13]:
togepi.level = 19

19

In [14]:
togepi

m_Pokemon("Togepi", 19, 12.5)

We can also add type parameters to our structs to force an attribute to conform to the type specified by our code. That way, the user has the freedom to initially specify which type the particular struct will accept. 

In [15]:
# We parametrize the `level` argument so we can specify its data type manually
mutable struct mp_Pokemon{A}
    name::String
    level::A
    power::Float64
end  

In [16]:
# This will result in an error because `level` should be an Int64-type
charmander = mp_Pokemon{Int64}("Charmander",54.5,32.1)

LoadError: InexactError: Int64(54.5)

In [17]:
charmander = mp_Pokemon{Int64}("Charmander",54,32.1)

mp_Pokemon{Int64}("Charmander", 54, 32.1)

# KR3: Type Inference
Julia is able to infer the data type of a container variable (e.g. array, dictionary, etc.) even if the elements inside it have not been explicitly determined

In [18]:
Array(range(2,5,step=1))

4-element Vector{Int64}:
 2
 3
 4
 5

In [19]:
Array(range(2,5,step=0.5))

7-element Vector{Float64}:
 2.0
 2.5
 3.0
 3.5
 4.0
 4.5
 5.0

In [20]:
# Julia will automatically infer that a range of integers would be Int64. If we want to regard them as floats instead, we can specify them in curly braces
Array{Float64}(range(2,5,step=1))

4-element Vector{Float64}:
 2.0
 3.0
 4.0
 5.0

# KR4: Type instability
As Julia compiles functions, it can infer the output datatypes given an input type. If only the input type affects the output type, we can say that the function has type-stability

In [21]:
# This is a function that intentionally induces type instability
function lvlup(lvl)
    if lvl > 100
        return "Already at max level"
    else
        return lvl += 1
    end
end

lvlup (generic function with 1 method)

In [22]:
typeof(lvlup(34))

Int64

In [23]:
typeof(lvlup(104))

String

Even if both inputs are of `Int64` data type, the outputs can either be `Int64` or `String` based on the input value. This function is therefore type-unstable. 
One way to fix this is to manually alter the function so that it would always output a `String` type

In [24]:
function s_lvlup(lvl)
    if lvl > 100
        return "Already at max level"
    else
        return string(lvl += 1)
    end
end

s_lvlup (generic function with 1 method)

In [25]:
typeof(s_lvlup(34))

String

# KR5: Detecting type instability
We can use the `@code_warntype` macro do detemine type instability

In [26]:
@code_warntype lvlup(34)

Variables
  #self#::Core.Const(lvlup)
  lvl@_2::Int64
  lvl@_3::Int64

Body::Union{Int64, String}
1 ─      (lvl@_3 = lvl@_2)
│   %2 = (lvl@_3 > 100)::Bool
└──      goto #3 if not %2
2 ─      return "Already at max level"
3 ─ %5 = (lvl@_3 + 1)::Int64
│        (lvl@_3 = %5)
└──      return %5


We have known prior that `lvlup()` returns two types depending on its input. This is evident in the `@code_warntype` macro by showing that `Body:: ____` contains a `Union{}` type, meaning that these are the possible outputs of the given function.

# KR6: Arrays with abstract types
The elements in an array can have its type pre-determined. Here we create two similar arrays, with one specified with the abstract type `Number`, of which `Float64` is a subclass

In [27]:
floatMatrix = Float64[0.5,0.6,0.7,0.8]

4-element Vector{Float64}:
 0.5
 0.6
 0.7
 0.8

In [28]:
numberMatrix = Number[0.5,0.6,0.7,0.8]

4-element Vector{Number}:
 0.5
 0.6
 0.7
 0.8

In [29]:
function multiply(x::Array{T}) where T <: Number
    return x.*x
end

multiply (generic function with 1 method)

In [30]:
using BenchmarkTools

In [31]:
@benchmark multiply(floatMatrix)

BenchmarkTools.Trial: 10000 samples with 990 evaluations.
 Range (min … max):  50.101 ns …  1.875 μs  ┊ GC (min … max): 0.00% … 95.24%
 Time  (median):     52.626 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   60.094 ns ± 55.635 ns  ┊ GC (mean ± σ):  4.26% ±  4.57%

  ▄█▇▅▄▃▂▁▁       ▁▂▁▁▁  ▁▁▁▁▁▁                               ▁
  ███████████████████████████████▇▆▅▆▆▅▆▅▆▅▆▅▆▆▄▅▅▅▂▅▄▄▄▄▄▃▄▄ █
  50.1 ns      Histogram: log(frequency) by time       116 ns <

 Memory estimate: 112 bytes, allocs estimate: 1.

In [32]:
@benchmark multiply(numberMatrix)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.100 μs … 423.080 μs  ┊ GC (min … max): 0.00% … 99.35%
 Time  (median):     1.150 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.266 μs ±   4.226 μs  ┊ GC (mean ± σ):  3.32% ±  0.99%

  ▅█▅▆▄▂▄▅▃▃▂▁▃▃▂▃▁▂▁ ▁                                       ▂
  ██████████████████████▇██▇██▇▇▅▇▇▄▆▆▄▆▅▅▆▅▄▅▄▆▅▃▅▅▄▄▅▂▄▄▂▄▅ █
  1.1 μs       Histogram: log(frequency) by time      2.07 μs <

 Memory estimate: 560 bytes, allocs estimate: 13.

Using `@benchmark`, it can be seen that using higher-tier types will result in a slower runtime since Julia will still have to infer among the subtypes of that abstract type